In [85]:
from fairworkflows import mark_as_fairstep, FairWorkflow, FairStep
import functools
from noodles import schedule, run_single, run_parallel
from noodles.tutorial import display_workflows


def fairstep(func):
    func.fs = mark_as_fairstep(func)
    return schedule(func)

def fairworkflow(name=None):
    def fairworkflow_inner(func):
        def wrapper(*args, **kwargs):
            print(*args, **kwargs)
            promise = func(*args, **kwargs)
            print('aaa', promise)
            class FairWorkflowP:
                def __init__(self):
                    self.name = name
                    self.promise = promise
                def display(self):
                    display_workflows(prefix='control', workflow=self.promise)
                def execute(self, num_threads=1):
                    if num_threads==1:
                        return run_single(self.promise)
                    elif num_threads>1:
                        return run_parallel(self.promise, num_threads)
            return FairWorkflowP()
        return wrapper
    return fairworkflow_inner

In [86]:
@fairstep
def add(a, b):
  """Adding up numbers!"""
  return a + b

In [87]:
@fairstep
def sub(a, b):
  """Subtracting numbers."""
  return a - b

In [88]:
@fairstep
def mul(a, b):
  """Multiplying numbers."""
  return a * b

In [89]:
@fairworkflow(name='My Workflow')
def my_workflow(in1, in2, in3):
    t1 = add(in1, in2)
    t2 = sub(in1, in2)
    t3 = mul(sub(t1, in3), t2)
    return t3

In [90]:
fw = my_workflow(1, 4, 3)

1 4 3
aaa <noodles.interface.decorator.PromisedObject object at 0x7f801588f700>


In [91]:
fw.display()

| workflow |
| --- |
| ![workflow workflow](control-workflow.svg) |

In [92]:
answer = fw.execute(num_threads=2)

answer

-6